CNN(합성곱 신경망)

- 이미지나 비디오 같은 영상 인식에 특화된 설계로, 병렬 처리가 쉬워 대규모 서비스에 적용가능
- 최근 이미지뿐 아니라 자연어 처리 추천 시스템에 응용되기도 한다. 

1) 컴퓨터가 보는 이미지 

- 컴퓨터에서 모든 이미지는 픽셀값들을 가로, 세로로 늘어놓은 행렬로 표현할 수 있다. 
- 일반적인 인공신경망은 다양한 형태의 입력에 대한 확장성이 떨어진다. 


2) 컨볼루션

- 계층적으로 이미지를 인식할 수 있도록 단계마다 이미지의 특징을 추출 하는 것.
- 각 단계에서는 이미지에 다양한 필터를 적용하여 윤곽선, 질감, 털 등 각종 특징 추출.
- 필터 적용시 이미지 왼쪽 위에서 오른쪽 밑까지 밀어가며 곱하고 더하는데 이 작업을 '컨볼루션'이라고 한다. 
- 컨볼루션은 모든 종류의 이미지로 확장하기 어렵고, 사람의 실력에 따라 모델 성능이 달라지고, 일이리 작업하기에는 시간과 비용이 크다. 
- CNN은 이미지를 추출하는 필터를 학습한다. ( 필터 하나가 작은 신경망 )

3) CNN모델

- 일반적으로 컨볼루션 계층, 풀링 계층, 특징들을 모아 최종 분류하는 일반적인 인공 신경망 계층으로 구성
- 컨볼루션 계층은 이미지의 특징을 추출하는 역할, 풀링은 필터를 거친 여러 특징 중 중요한 특징 하나를 골라냄( 나머지는 특징은 버려 차원이 감소)
- 풀고자 하는 문제에 따라 계층 구성을 달리 할 수 있으며, 컨볼루션계층만으로 구성된 모델을 만들 수도 있다. 
- 컨볼루션 연산은 이미지를 겹치는 매우 작은 조각으로 쪼개어 필터 기능을 하는 작은 신경망에 적용 ( 컨볼루션 필터 or 커널 )
- 컨볼루션은 오른쪽 아래로 움직이며 이미지를 만든다. 이때 움직임을 조절하는 값을 스트라이드라고 한다. 
- 컨볼루션을 거쳐 만들어진 새로운 이미지는 특징 맵이라고도 부른다.  이 특징 맵들이 풀링 게층으로 넘어간다. 
- 특징 맵의 크기가 크면 학습이 어렵고, 과적합의 위험이 증가한다. 
- CNN은 사물의 치우침에 따라 성능이 변하는 인공신경망의 문제를 해결해주고, 이미지 크기만큼 가중치를 가져야 하는

  일반 인공 신경망과는 다르게 필터만을 학습시키면 되어 훨씬 적은 계산량으로 효율적인 학습 가능

In [23]:
#CNN모델 구현 
#컨볼루션 -> 풀링 -> 드롭아웃 -> 풀링 -> 신경망 -> 드롭아웃 -> 신경망
#CNN모델의 커널 크기는 5*5 / 컨볼루션 계층은 2개 

#라이브러리 로드 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

#쿠다 사용환경 설정
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

#하이퍼파라미터 설정
EPOCHS = 40
BATCH_SIZE = 64

#예제 데이터셋 로드
#transforms 이용 전처리는 파이토치 텐서화, 정규화만 적용

train_loader = torch.utils.data.DataLoader(datasets.FashionMNIST('./.data',
                                                                train = True,
                                                                download = True, 
                                                                transform = transforms.Compose([transforms.ToTensor(),
                                                                                               transforms.Normalize((0.1307,),(0.3081,))
                                                                                               ]))
                                          , batch_size = BATCH_SIZE, shuffle = True)

test_loader = torch.utils.data.DataLoader(datasets.FashionMNIST('./.data',
                                                                train = False,
                                                                download = True, 
                                                                transform = transforms.Compose([transforms.ToTensor(),
                                                                                               transforms.Normalize((0.1307,),(0.3081,))
                                                                                               ]))
                                          , batch_size = BATCH_SIZE, shuffle = True)



In [18]:
#nn.Conv2d모듈은 입력 x를 받는 함수를 반환한다 ( 자신을 바로 부를 수 잇지만 함수로 생각해도 무방)
#모델의 학습 정의
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #입력채널수 = 1, 출력 채널수 =10 ( 데이터셋이 흑백이미지로 색상 채널이 1개뿐)
        #kernel_size로 커널 크기 지정 숫자 하나만 지저앟면 정사각형으로 간주
        self.conv1 = nn.Conv2d(1,10, kernel_size = 5)
        self.conv2 = nn.Conv2d(10,20, kernel_size = 5)
        #컨볼루션 결과로 나온 출력값에 드롭아웃 
        self.drop = nn.Dropout2d()
        #일반신경망
        #위 계층의 출력값인 320을 입력 받고 
        #최종 출력을 분류할 클래스 개수인 10개로 설정
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
        
    #출력까지 진행
    def forward(self, x):
        #두개의 각 컨볼루션 계층을 거친 후, max_pool2d 함수 거치기 / 두번째 입력은 커널 크기
        #컨볼루션과 맥스 풀링을 통과한 x는 F.relu함수를 거친다. 
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x),2))
        #2차원의 특징맵을 바로 입력으로 넣을 수 없어 1차원으로 변환(축소)
        #view()의 첫 입력 -1은 '남는 모든 차원', 320은 x가 가진 원소갯수
        x = x.view(-1, 320)
        #앞서 추출한 특징들을 입력으로 받아 분류하는 신경망 계층 구성( 1- ReLu활성화함수 2- 드롭아웃, 3- 0~9레이블)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)

In [19]:
#`to()` 함수는 모델의 파라미터들을 지정한 곳으로 보내는 역할을 한다. 일반적으로 CPU 1개만 사용할 경우 필요는 없지만, GPU를 사용하고자 하는 경우 `to("cuda")`로 지정하여 GPU로 보내야 한다.
# 최적화 알고리즘으로는 토치에 내장되어 있는 `optim.SGD`를 사용(확률적 경사하강법)

model     = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [20]:
#모델 train모드, F.corss_entropy오차 함수를 이용하여 모델 출력과 정답인 타겟값 사이 오차 계산
#역전파 알고리즘을 실행해주는 loss.backward()함수를 이용해 기울기 계산 후, optimizer.step() 최적화 함수로 구한 기울기값으로 학습 파라미터 갱신

def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
         #epoch마다 새로운 경사값을 계산하므로 zero_grad()함수를 호출해 경사를 0으로 설정
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output ,target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0:
            print('Train Epoch : {} [{}/{} ({}%)]\t Loss :{}'.format
                 (epoch ,batch_idx*len(data), len(train_loader.dataset), 100.*batch_idx/len(train_loader), loss.item()))
            


In [21]:
#성능 확인

def evaluate(model, test_loader):
    model.eval()
    #초기값 설정
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            #배치 오차 합산
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            #가장 높은 값을 가진 인덱스가 예측값
            pred = output.max(1, keepdim = True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100* correct / len(test_loader.dataset)
    return test_loss, test_accuracy    

In [24]:
#학습
for epoch in range(1, EPOCHS +1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test LOSS : {}, Accuracy : {}'.format(epoch, test_loss, test_accuracy))

Train Epoch : 1 [0/60000 (0.0%)]	 Loss :4.606046676635742
Train Epoch : 1 [12800/60000 (21.321961620469082%)]	 Loss :0.9683188199996948
Train Epoch : 1 [25600/60000 (42.643923240938165%)]	 Loss :0.759576678276062
Train Epoch : 1 [38400/60000 (63.96588486140725%)]	 Loss :0.7466010451316833
Train Epoch : 1 [51200/60000 (85.28784648187633%)]	 Loss :0.9223911166191101
[1] Test LOSS : 0.5916794527053832, Accuracy : 77.55
Train Epoch : 2 [0/60000 (0.0%)]	 Loss :0.7344424724578857
Train Epoch : 2 [12800/60000 (21.321961620469082%)]	 Loss :0.600426435470581
Train Epoch : 2 [25600/60000 (42.643923240938165%)]	 Loss :0.7209107875823975
Train Epoch : 2 [38400/60000 (63.96588486140725%)]	 Loss :0.5604140758514404
Train Epoch : 2 [51200/60000 (85.28784648187633%)]	 Loss :0.5895727872848511
[2] Test LOSS : 0.5129018677711487, Accuracy : 81.52
Train Epoch : 3 [0/60000 (0.0%)]	 Loss :0.4853051006793976
Train Epoch : 3 [12800/60000 (21.321961620469082%)]	 Loss :0.5364071130752563
Train Epoch : 3 [25600

인공 신경망과 드롭아웃 조합보다 약 1%이상이 개선되었다. 